In [1]:
!pip install -qqq openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00


In [3]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('openai-key')

In [7]:
import pandas as pd
import random

# Load the CSV file
file_path = 'clickbait_data.csv'
df = pd.read_csv(file_path,encoding='latin-1')

# Ensure the dataset has the expected structure
assert 'Title' in df.columns and 'Label' in df.columns, "CSV must contain 'title' and 'label' columns"

# Separate clickbait and non-clickbait titles
clickbait = df[df['Label'] == 1]
non_clickbait = df[df['Label'] == 0]

# Randomly sample 200 clickbait and 200 non-clickbait titles for fine-tuning
clickbait_sample = clickbait.sample(n=200, random_state=42)
non_clickbait_sample = non_clickbait.sample(n=200, random_state=42)

# Combine samples for fine-tuning
fine_tuning_data = pd.concat([clickbait_sample, non_clickbait_sample])

# Save fine-tuning data to a CSV file
fine_tuning_data.to_csv('fine_tuning_data.csv', index=False)


In [8]:
from openai import OpenAI
client = OpenAI()

In [16]:

import json

# Load the CSV file
file_path = 'clickbait_data.csv'
df = pd.read_csv(file_path)

# Ensure the dataset has the expected structure
assert 'Title' in df.columns and 'Label' in df.columns, "CSV must contain 'Title' and 'Label' columns"

# Separate clickbait and non-clickbait titles
clickbait = df[df['Label'] == 1]
non_clickbait = df[df['Label'] == 0]

# Randomly sample 200 clickbait and 200 non-clickbait titles for fine-tuning
clickbait_sample = clickbait.sample(n=200, random_state=42)
non_clickbait_sample = non_clickbait.sample(n=200, random_state=42)

# Combine samples for fine-tuning
fine_tuning_data = pd.concat([clickbait_sample, non_clickbait_sample])

# Convert the fine-tuning data to JSONL format
jsonl_file_path = 'fine_tuning_data.jsonl'

with open(jsonl_file_path, 'w') as jsonl_file:
    for _, row in fine_tuning_data.iterrows():
        record = {
            "messages": [
                {"role": "system", "content": "You are a helpful assistant to tell if a title is click-baiting or not"},
                {"role": "user", "content": str(row['Title'])},
                {"role": "assistant", "content": str(row['Label'])}
            ]
        }
        jsonl_file.write(json.dumps(record) + '\n')

print(f"Fine-tuning data has been saved as {jsonl_file_path}")




Fine-tuning data has been saved as fine_tuning_data.jsonl


In [18]:
with open('fine_tuning_data.jsonl', 'rb') as file:
    result = client.files.create(
        file=file,
        purpose='fine-tune'
    )

# Get the file ID
file_id = result.id

print(f"File ID: {file_id}")

File ID: file-EQ904JLwcHUrWn2yf1samy7p


In [19]:

client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-ZBW2oMw6RStB1u8R3peV0ud4', created_at=1719871087, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-sQBgvbA6gg6Nq6d8ze0NO27y', result_files=[], seed=2026029894, status='validating_files', trained_tokens=None, training_file='file-EQ904JLwcHUrWn2yf1samy7p', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [20]:

client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-ZBW2oMw6RStB1u8R3peV0ud4', created_at=1719871087, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-sQBgvbA6gg6Nq6d8ze0NO27y', result_files=[], seed=2026029894, status='validating_files', trained_tokens=None, training_file='file-EQ904JLwcHUrWn2yf1samy7p', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-oaqLimsSL5M0QWFilT5EhBBt', created_at=1719867850, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Unexpected file format, expected either prompt/completion pairs or chat messages.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch

In [33]:
# Randomly sample 200 clickbait and 200 non-clickbait titles for testing
clickbait_test_sample = clickbait.sample(n=300, random_state=21)
non_clickbait_test_sample = non_clickbait.sample(n=300, random_state=21)

# Combine samples for testing
testing_data = pd.concat([clickbait_test_sample, non_clickbait_test_sample])

# Test the fine-tuned model and evaluate accuracy
correct_predictions = 0

for index, row in testing_data.iterrows():
    prompt = row['Title']
    #prompt="if the {prompt} title is click-baiting then output 1 otherwise output 0. Nothing else is to be output."
    actual_label = row['Label']

    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:cstamucc::9gJuUhqo",
        messages=[
        {"role": "system", "content":"You are a helpful assistant to tell if a title is clickbait or not" },
        {"role": "user", "content": prompt}
  ],
        temperature=0
    )

    predicted_label = int(response.choices[0].message.content)
    print(predicted_label)
    if predicted_label == actual_label:
        correct_predictions += 1

accuracy = correct_predictions / len(testing_data)
print(f'Accuracy: {accuracy * 100:.2f}%')


1
1
1
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 